MovieLens Data Analysys:

MovieLens data sets were collected by the GroupLens Research Project at the University of Minnesota. This data set consists of 100,000 ratings (1-5) from 943 users on 1682 movies. Each user has rated at least 20 movies. Simple demographic info for the users (age, gender, occupation, zip).

Check for the availability of Spark Context in Data Bricks Cluster

In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

Import necessary library for running spark application

In [0]:

import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import desc
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import Window
from pyspark.sql.functions import rank,lit,column,row_number

Here load u.item and attach a schema to the data set.

u.item     -- Information about the items (movies); this is a tab separated
              list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.

In [0]:
schema = StructType([StructField("movieId",IntegerType()),
                         StructField("title",StringType()),
                         StructField("releaseDate",StringType()),
                         StructField("videoReleaseDate",StringType()),
                         StructField("URL",StringType()),
                         StructField("unknown",StringType()),
                         StructField("action",StringType()),
                         StructField("adventure",StringType()),
                         StructField("animation",StringType()),
                         StructField("children",StringType()),
                         StructField("comedy",StringType()),
                         StructField("crime",StringType()),
                         StructField("documentary",StringType()),
                         StructField("drama",StringType()),
                         StructField("fantasy",StringType()),
                         StructField("filmnoir",StringType()),
                         StructField("horror",StringType()),
                         StructField("musical",StringType()),
                         StructField("mistory",StringType()),
                         StructField("romance",StringType()),
                         StructField("SciFi",StringType()),
                         StructField("thriller",StringType()),
                         StructField("war",StringType()),
                         StructField("western",StringType())])

item_df = spark.read.schema(schema).load("/FileStore/tables/u.item", format = "csv", sep = "|", header = "false")

In [0]:
item_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- releaseDate: string (nullable = true)
 |-- videoReleaseDate: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- unknown: string (nullable = true)
 |-- action: string (nullable = true)
 |-- adventure: string (nullable = true)
 |-- animation: string (nullable = true)
 |-- children: string (nullable = true)
 |-- comedy: string (nullable = true)
 |-- crime: string (nullable = true)
 |-- documentary: string (nullable = true)
 |-- drama: string (nullable = true)
 |-- fantasy: string (nullable = true)
 |-- filmnoir: string (nullable = true)
 |-- horror: string (nullable = true)
 |-- musical: string (nullable = true)
 |-- mistory: string (nullable = true)
 |-- romance: string (nullable = true)
 |-- SciFi: string (nullable = true)
 |-- thriller: string (nullable = true)
 |-- war: string (nullable = true)
 |-- western: string (nullable = true)



In [0]:
item_df.show()

+-------+--------------------+-----------+----------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|movieId|               title|releaseDate|videoReleaseDate|                 URL|unknown|action|adventure|animation|children|comedy|crime|documentary|drama|fantasy|filmnoir|horror|musical|mistory|romance|SciFi|thriller|war|western|
+-------+--------------------+-----------+----------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|      1|    Toy Story (1995)|01-Jan-1995|            null|http://us.imdb.co...|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|       0|     0|      0|      0|      0|    0|       0|  0|      0|
|      2|    GoldenEye (1995)|01-Jan-1995|            null|http://us.imdb.co

Here transform the data frame to a usable format for further data analysis operations.

In [0]:
item_df1 = item_df.drop('videoReleaseDate','URL')

In [0]:
item_df1.display()

movieId,title,releaseDate,unknown,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mistory,romance,SciFi,thriller,war,western
1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),01-Jan-1995,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,Twelve Monkeys (1995),01-Jan-1995,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
8,Babe (1995),01-Jan-1995,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
9,Dead Man Walking (1995),01-Jan-1995,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
10,Richard III (1995),22-Jan-1996,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [0]:
item_df1 = item_df1.drop('releaseDate')
item_df1.display()

movieId,title,unknown,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mistory,romance,SciFi,thriller,war,western
1,Toy Story (1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,Twelve Monkeys (1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
8,Babe (1995),0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
9,Dead Man Walking (1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
10,Richard III (1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [0]:
item_df1.show()

+-------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|movieId|               title|unknown|action|adventure|animation|children|comedy|crime|documentary|drama|fantasy|filmnoir|horror|musical|mistory|romance|SciFi|thriller|war|western|
+-------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|      1|    Toy Story (1995)|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|       0|     0|      0|      0|      0|    0|       0|  0|      0|
|      2|    GoldenEye (1995)|      0|     1|        1|        0|       0|     0|    0|          0|    0|      0|       0|     0|      0|      0|      0|    0|       1|  0|      0|
|      3|   Four Rooms (1995)|      0|     0|        0|        0|       0|     0|    0|        

In [0]:
item_df2 = item_df1.withColumn("unknown",f.when(f.col("unknown")==f.lit("1"),f.lit("unknown")).otherwise("null")).withColumn("action",f.when(f.col("action")==f.lit("1"),f.lit("action")).otherwise("null")).withColumn("adventure",f.when(f.col("adventure")==f.lit("1"),f.lit("adventure")).otherwise("null")).withColumn("animation",f.when(f.col("animation")==f.lit("1"),f.lit("animation")).otherwise("null")).withColumn("children",f.when(f.col("children")==f.lit("1"),f.lit("children")).otherwise("null")).withColumn("comedy",f.when(f.col("comedy")==f.lit("1"),f.lit("comedy")).otherwise("null")).withColumn("crime",f.when(f.col("crime")==f.lit("1"),f.lit("crime")).otherwise("null")).withColumn("documentary",f.when(f.col("documentary")==f.lit("1"),f.lit("documentary")).otherwise("null")).withColumn("drama",f.when(f.col("drama")==f.lit("1"),f.lit("drama")).otherwise("null")).withColumn("fantasy",f.when(f.col("fantasy")==f.lit("1"),f.lit("fantasy")).otherwise("null")).withColumn("filmnoir",f.when(f.col("filmnoir")==f.lit("1"),f.lit("filmnoir")).otherwise("null")).withColumn("horror",f.when(f.col("horror")==f.lit("1"),f.lit("horror")).otherwise("null")).withColumn("musical",f.when(f.col("musical")==f.lit("1"),f.lit("musical")).otherwise("null")).withColumn("mistory",f.when(f.col("mistory")==f.lit("1"),f.lit("mistory")).otherwise("null")).withColumn("romance",f.when(f.col("romance")==f.lit("1"),f.lit("romance")).otherwise("null")).withColumn("SciFi",f.when(f.col("SciFi")==f.lit("1"),f.lit("SciFi")).otherwise("null")).withColumn("thriller",f.when(f.col("thriller")==f.lit("1"),f.lit("thriller")).otherwise("null")).withColumn("war",f.when(f.col("war")==f.lit("1"),f.lit("war")).otherwise("null")).withColumn("western",f.when(f.col("western")==f.lit("1"),f.lit("western")).otherwise("null"))


In [0]:
item_df2.display()

movieId,title,unknown,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mistory,romance,SciFi,thriller,war,western
1,Toy Story (1995),null,null,null,animation,children,comedy,null,null,null,null,null,null,null,null,null,null,null,null,null
2,GoldenEye (1995),null,action,adventure,null,null,null,null,null,null,null,null,null,null,null,null,null,thriller,null,null
3,Four Rooms (1995),null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,thriller,null,null
4,Get Shorty (1995),null,action,null,null,null,comedy,null,null,drama,null,null,null,null,null,null,null,null,null,null
5,Copycat (1995),null,null,null,null,null,null,crime,null,drama,null,null,null,null,null,null,null,thriller,null,null
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),null,null,null,null,null,null,null,null,drama,null,null,null,null,null,null,null,null,null,null
7,Twelve Monkeys (1995),null,null,null,null,null,null,null,null,drama,null,null,null,null,null,null,SciFi,null,null,null
8,Babe (1995),null,null,null,null,children,comedy,null,null,drama,null,null,null,null,null,null,null,null,null,null
9,Dead Man Walking (1995),null,null,null,null,null,null,null,null,drama,null,null,null,null,null,null,null,null,null,null
10,Richard III (1995),null,null,null,null,null,null,null,null,drama,null,null,null,null,null,null,null,null,war,null


In [0]:
itemDF2 = item_df2.withColumn("genre",f.concat_ws(",",f.col("unknown"),f.col("action"),f.col("adventure"),f.col("animation"),f.col("children"),f.col("comedy"),
                                                  f.col("crime"),f.col("documentary"),f.col("drama"),f.col("fantasy"),f.col("filmnoir"),f.col("horror"),
                                                  f.col("musical"),f.col("mistory"),f.col("romance"),f.col("SciFi"),f.col("thriller"),f.col("war"),f.col("western")))
itemDF3 = itemDF2.withColumn("genre",f.split(f.col("genre"),",")) 
itemDF4 = itemDF3.select(itemDF3.movieId,itemDF3.title,f.explode(itemDF3.genre).alias("genre")) 
itemDF5 = itemDF4.filter(f.col("genre") != "null")
itemDF5.show()



+-------+--------------------+---------+
|movieId|               title|    genre|
+-------+--------------------+---------+
|      1|    Toy Story (1995)|animation|
|      1|    Toy Story (1995)| children|
|      1|    Toy Story (1995)|   comedy|
|      2|    GoldenEye (1995)|   action|
|      2|    GoldenEye (1995)|adventure|
|      2|    GoldenEye (1995)| thriller|
|      3|   Four Rooms (1995)| thriller|
|      4|   Get Shorty (1995)|   action|
|      4|   Get Shorty (1995)|   comedy|
|      4|   Get Shorty (1995)|    drama|
|      5|      Copycat (1995)|    crime|
|      5|      Copycat (1995)|    drama|
|      5|      Copycat (1995)| thriller|
|      6|Shanghai Triad (Y...|    drama|
|      7|Twelve Monkeys (1...|    drama|
|      7|Twelve Monkeys (1...|    SciFi|
|      8|         Babe (1995)| children|
|      8|         Babe (1995)|   comedy|
|      8|         Babe (1995)|    drama|
|      9|Dead Man Walking ...|    drama|
+-------+--------------------+---------+
only showing top

In [0]:
itemDF6 = itemDF5.select('movieId','title','genre',f.regexp_extract('title',r'\((\d+)\)',1).alias('year'))
itemDF6.show()

+-------+--------------------+---------+----+
|movieId|               title|    genre|year|
+-------+--------------------+---------+----+
|      1|    Toy Story (1995)|animation|1995|
|      1|    Toy Story (1995)| children|1995|
|      1|    Toy Story (1995)|   comedy|1995|
|      2|    GoldenEye (1995)|   action|1995|
|      2|    GoldenEye (1995)|adventure|1995|
|      2|    GoldenEye (1995)| thriller|1995|
|      3|   Four Rooms (1995)| thriller|1995|
|      4|   Get Shorty (1995)|   action|1995|
|      4|   Get Shorty (1995)|   comedy|1995|
|      4|   Get Shorty (1995)|    drama|1995|
|      5|      Copycat (1995)|    crime|1995|
|      5|      Copycat (1995)|    drama|1995|
|      5|      Copycat (1995)| thriller|1995|
|      6|Shanghai Triad (Y...|    drama|1995|
|      7|Twelve Monkeys (1...|    drama|1995|
|      7|Twelve Monkeys (1...|    SciFi|1995|
|      8|         Babe (1995)| children|1995|
|      8|         Babe (1995)|   comedy|1995|
|      8|         Babe (1995)|    

Creat dataframe with decade field. As every deacade has the first three places are same, here extract the first three places of year column value.

In [0]:

itemDF7 = itemDF6.withColumn('decade',f.substring('year',1,3))
itemDF7.show()

+-------+--------------------+---------+----+------+
|movieId|               title|    genre|year|decade|
+-------+--------------------+---------+----+------+
|      1|    Toy Story (1995)|animation|1995|   199|
|      1|    Toy Story (1995)| children|1995|   199|
|      1|    Toy Story (1995)|   comedy|1995|   199|
|      2|    GoldenEye (1995)|   action|1995|   199|
|      2|    GoldenEye (1995)|adventure|1995|   199|
|      2|    GoldenEye (1995)| thriller|1995|   199|
|      3|   Four Rooms (1995)| thriller|1995|   199|
|      4|   Get Shorty (1995)|   action|1995|   199|
|      4|   Get Shorty (1995)|   comedy|1995|   199|
|      4|   Get Shorty (1995)|    drama|1995|   199|
|      5|      Copycat (1995)|    crime|1995|   199|
|      5|      Copycat (1995)|    drama|1995|   199|
|      5|      Copycat (1995)| thriller|1995|   199|
|      6|Shanghai Triad (Y...|    drama|1995|   199|
|      7|Twelve Monkeys (1...|    drama|1995|   199|
|      7|Twelve Monkeys (1...|    SciFi|1995| 

Read u.data file.

u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly
              ordered. This is a tab separated list of 
	         user id | item id | rating | timestamp.

As here data is not in a appropiet dataframe format apply necessary transformation to make it more cleaner and insightful.

In [0]:
#read u.data file
dataDF = spark.read.text("/FileStore/tables/u.data")
dataDF = dataDF.withColumn("splitable",f.split("value","\t"))
dataDF.show(15,False)

+-----------------------+-------------------------+
|value                  |splitable                |
+-----------------------+-------------------------+
|196\t242\t3\t881250949 |[196, 242, 3, 881250949] |
|186\t302\t3\t891717742 |[186, 302, 3, 891717742] |
|22\t377\t1\t878887116  |[22, 377, 1, 878887116]  |
|244\t51\t2\t880606923  |[244, 51, 2, 880606923]  |
|166\t346\t1\t886397596 |[166, 346, 1, 886397596] |
|298\t474\t4\t884182806 |[298, 474, 4, 884182806] |
|115\t265\t2\t881171488 |[115, 265, 2, 881171488] |
|253\t465\t5\t891628467 |[253, 465, 5, 891628467] |
|305\t451\t3\t886324817 |[305, 451, 3, 886324817] |
|6\t86\t3\t883603013    |[6, 86, 3, 883603013]    |
|62\t257\t2\t879372434  |[62, 257, 2, 879372434]  |
|286\t1014\t5\t879781125|[286, 1014, 5, 879781125]|
|200\t222\t5\t876042340 |[200, 222, 5, 876042340] |
|210\t40\t3\t891035994  |[210, 40, 3, 891035994]  |
|224\t29\t3\t888104457  |[224, 29, 3, 888104457]  |
+-----------------------+-------------------------+
only showing

In [0]:
dataDF= dataDF.withColumn("userId",f.col("splitable")[0].cast('integer')).withColumn("itemId",f.col("splitable")[1].cast('integer')).withColumn("rating",f.col("splitable")[2].cast('integer')).withColumn("timestamp",f.col("splitable")[3].cast('integer'))
dataDF.show(15,False)

+-----------------------+-------------------------+------+------+------+---------+
|value                  |splitable                |userId|itemId|rating|timestamp|
+-----------------------+-------------------------+------+------+------+---------+
|196\t242\t3\t881250949 |[196, 242, 3, 881250949] |196   |242   |3     |881250949|
|186\t302\t3\t891717742 |[186, 302, 3, 891717742] |186   |302   |3     |891717742|
|22\t377\t1\t878887116  |[22, 377, 1, 878887116]  |22    |377   |1     |878887116|
|244\t51\t2\t880606923  |[244, 51, 2, 880606923]  |244   |51    |2     |880606923|
|166\t346\t1\t886397596 |[166, 346, 1, 886397596] |166   |346   |1     |886397596|
|298\t474\t4\t884182806 |[298, 474, 4, 884182806] |298   |474   |4     |884182806|
|115\t265\t2\t881171488 |[115, 265, 2, 881171488] |115   |265   |2     |881171488|
|253\t465\t5\t891628467 |[253, 465, 5, 891628467] |253   |465   |5     |891628467|
|305\t451\t3\t886324817 |[305, 451, 3, 886324817] |305   |451   |3     |886324817|
|6\t

In [0]:
dataDF = dataDF.select("userId","itemId","rating","timestamp")
dataDF.show(15,False)

+------+------+------+---------+
|userId|itemId|rating|timestamp|
+------+------+------+---------+
|196   |242   |3     |881250949|
|186   |302   |3     |891717742|
|22    |377   |1     |878887116|
|244   |51    |2     |880606923|
|166   |346   |1     |886397596|
|298   |474   |4     |884182806|
|115   |265   |2     |881171488|
|253   |465   |5     |891628467|
|305   |451   |3     |886324817|
|6     |86    |3     |883603013|
|62    |257   |2     |879372434|
|286   |1014  |5     |879781125|
|200   |222   |5     |876042340|
|210   |40    |3     |891035994|
|224   |29    |3     |888104457|
+------+------+------+---------+
only showing top 15 rows



read u.user file.

u.user     -- Demographic information about the users; this is a tab
              separated list of
              user id | age | gender | occupation | zip code
              The user ids are the ones used in the u.data data set.

In [0]:
#read u.user file
userSchema = StructType([StructField("userId",IntegerType()),
                         StructField("age",IntegerType()),
                         StructField("gender",StringType()),
                         StructField("occupation",StringType()), 
                         StructField("zipCode",IntegerType())])


userDF = spark.read.schema(userSchema).load("/FileStore/tables/u.user", format = "csv", sep = "|", header = "false")
userDF.show(15,False)

+------+---+------+-------------+-------+
|userId|age|gender|occupation   |zipCode|
+------+---+------+-------------+-------+
|1     |24 |M     |technician   |85711  |
|2     |53 |F     |other        |94043  |
|3     |23 |M     |writer       |32067  |
|4     |24 |M     |technician   |43537  |
|5     |33 |F     |other        |15213  |
|6     |42 |M     |executive    |98101  |
|7     |57 |M     |administrator|91344  |
|8     |36 |M     |administrator|5201   |
|9     |29 |M     |student      |1002   |
|10    |53 |M     |lawyer       |90703  |
|11    |39 |F     |other        |30329  |
|12    |28 |F     |other        |6405   |
|13    |47 |M     |educator     |29206  |
|14    |45 |M     |scientist    |55106  |
|15    |49 |F     |educator     |97301  |
+------+---+------+-------------+-------+
only showing top 15 rows



Highest rated movies by genre.

In [0]:
#Join item and data file  on id column which is common for both data frames
joinDF = itemDF5.join(dataDF,itemDF5.movieId == dataDF.itemId).drop(dataDF.itemId)
joinDF.show(15,False)

+-------+---------------------------------------------------------------------------+--------+------+------+---------+
|movieId|title                                                                      |genre   |userId|rating|timestamp|
+-------+---------------------------------------------------------------------------+--------+------+------+---------+
|242    |Kolya (1996)                                                               |comedy  |196   |3     |881250949|
|302    |L.A. Confidential (1997)                                                   |thriller|186   |3     |891717742|
|302    |L.A. Confidential (1997)                                                   |mistory |186   |3     |891717742|
|302    |L.A. Confidential (1997)                                                   |filmnoir|186   |3     |891717742|
|302    |L.A. Confidential (1997)                                                   |crime   |186   |3     |891717742|
|377    |Heavyweights (1994)                    

In [0]:


filterDF = joinDF.withColumn("rank",row_number().over(Window.partitionBy("genre").orderBy(f.col("rating").desc())))
filterDF.show()

+-------+--------------------+------+------+------+---------+----+
|movieId|               title| genre|userId|rating|timestamp|rank|
+-------+--------------------+------+------+------+---------+----+
|    222|Star Trek: First ...|action|   200|     5|876042340|   1|
|    234|         Jaws (1975)|action|   160|     5|876861185|   2|
|    201| Evil Dead II (1987)|action|   246|     5|884921594|   3|
|    241|Last of the Mohic...|action|   249|     5|879641194|   4|
|      4|   Get Shorty (1995)|action|    99|     5|886519097|   5|
|    181|Return of the Jed...|action|    25|     5|885853415|   6|
|    229|Star Trek III: Th...|action|   127|     5|884364867|   7|
|    144|     Die Hard (1988)|action|   291|     5|874835091|   8|
|    328|Conspiracy Theory...|action|   166|     5|886397722|   9|
|    174|Raiders of the Lo...|action|   160|     5|876860807|  10|
|     96|Terminator 2: Jud...|action|    42|     5|881107178|  11|
|    195|Terminator, The (...|action|    44|     5|878347874| 

In [0]:
filterDF1 = filterDF.filter(filterDF.rank==1)
filterDF1.show()

+-------+--------------------+-----------+------+------+---------+----+
|movieId|               title|      genre|userId|rating|timestamp|rank|
+-------+--------------------+-----------+------+------+---------+----+
|    222|Star Trek: First ...|     action|   200|     5|876042340|   1|
|    465|Jungle Book, The ...|  adventure|   253|     5|891628467|   1|
|     95|      Aladdin (1992)|  animation|    38|     5|892430094|   1|
|    327|     Cop Land (1997)|      crime|   287|     5|875333916|   1|
|   1022|Fast, Cheap & Out...|documentary|   146|     5|891458193|   1|
|    423|E.T. the Extra-Te...|    fantasy|    42|     5|881107687|   1|
|    673|    Cape Fear (1962)|   filmnoir|   200|     5|884128554|   1|
|    234|         Jaws (1975)|     horror|   160|     5|876861185|   1|
|     95|      Aladdin (1992)|    musical|    38|     5|892430094|   1|
|    267|             unknown|    unknown|   130|     5|875801239|   1|
|    387|Age of Innocence,...|      drama|   122|     5|87927045

Most dislike movies by genre.

In [0]:

filterDF = joinDF.withColumn("rank",row_number().over(Window.partitionBy("genre").orderBy("rating")))
filterDF2 = filterDF.filter(filterDF.rank==1)
filterDF2.show()


+-------+--------------------+-----------+------+------+---------+----+
|movieId|               title|      genre|userId|rating|timestamp|rank|
+-------+--------------------+-----------+------+------+---------+----+
|    232|   Young Guns (1988)|     action|   167|     1|892738341|   1|
|     82|Jurassic Park (1993)|  adventure|   219|     1|889452455|   1|
|   1240|Ghost in the Shel...|  animation|   279|     1|892174404|   1|
|    346| Jackie Brown (1997)|      crime|   166|     1|886397596|   1|
|     48|  Hoop Dreams (1994)|documentary|   161|     1|891170745|   1|
|   1336|       Kazaam (1996)|    fantasy|   279|     1|875298353|   1|
|    299|      Hoodlum (1997)|   filmnoir|    21|     1|874950931|   1|
|    288|       Scream (1996)|     horror|    20|     1|879667584|   1|
|    103|All Dogs Go to He...|    musical|    21|     1|874951245|   1|
|   1373| Good Morning (1971)|    unknown|   181|     1|878962052|   1|
|    346| Jackie Brown (1997)|      drama|   166|     1|88639759

Number of users rated movies by genre between age 40 - 45.

In [0]:

joinDF1 = userDF.join(joinDF,userDF.userId== joinDF.userId).drop(joinDF.userId)
joinDF1.show()

+------+---+------+----------+-------+-------+--------------------+---------+------+---------+
|userId|age|gender|occupation|zipCode|movieId|               title|    genre|rating|timestamp|
+------+---+------+----------+-------+-------+--------------------+---------+------+---------+
|   196| 49|     M|    writer|  55105|    242|        Kolya (1996)|   comedy|     3|881250949|
|   186| 39|     F| executive|      0|    302|L.A. Confidential...| thriller|     3|891717742|
|   186| 39|     F| executive|      0|    302|L.A. Confidential...|  mistory|     3|891717742|
|   186| 39|     F| executive|      0|    302|L.A. Confidential...| filmnoir|     3|891717742|
|   186| 39|     F| executive|      0|    302|L.A. Confidential...|    crime|     3|891717742|
|    22| 25|     M|    writer|  40206|    377| Heavyweights (1994)|   comedy|     1|878887116|
|    22| 25|     M|    writer|  40206|    377| Heavyweights (1994)| children|     1|878887116|
|   244| 28|     M|technician|  80525|     51|Lege

In [0]:
countUserDF = joinDF1.filter(f.col('age').between(40,45)).groupBy('genre').count().show()


+-----------+-----+
|      genre|count|
+-----------+-----+
|      crime|  639|
|    fantasy|  148|
|documentary|   66|
|     action| 2156|
|  animation|  316|
|     horror|  382|
|   filmnoir|  181|
|    musical|  509|
|  adventure| 1291|
|      drama| 3837|
|    western|  171|
|   children|  777|
|        war|  954|
|    mistory|  525|
|    romance| 1824|
|      SciFi| 1109|
|   thriller| 1840|
|     comedy| 2597|
+-----------+-----+



Count of movies by decade and genre.

In [0]:

countDFdecade = itemDF7.groupBy('decade','genre').count().show()

+------+---------+-----+
|decade|    genre|count|
+------+---------+-----+
|   197|  musical|    3|
|   193|    crime|    2|
|   198|   action|   31|
|   199|   horror|   58|
|   196|    drama|   19|
|   193|   comedy|   11|
|   193|   horror|    1|
|   194|animation|    4|
|   199|   action|  206|
|   195|adventure|    5|
|   199| filmnoir|    8|
|   197|adventure|    8|
|   199|      war|   40|
|   193|  mistory|    1|
|   194| thriller|    9|
|   195|    SciFi|    4|
|   194|  romance|    8|
|   198|    SciFi|   17|
|   193|  romance|    8|
|   192|    drama|    1|
+------+---------+-----+
only showing top 20 rows

